In [ ]:
import re
import os, sys
import random
import pickle
import numpy as np
import numpy.random
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from moviepy.editor import VideoFileClip, vfx
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Dense, Activation
from keras import regularizers
from keras.utils import to_categorical
from keras.utils import np_utils

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1818624/45929032 bytes (4.0%)5750784/45929032 bytes (12.5%)9658368/45929032 bytes (21.0%)13705216/45929032 bytes (29.8%)17801216/45929032 bytes (38.8%)21864448/45929032 bytes (47.6%)25829376/45929032 bytes (56.2%)29712384/45929032 bytes (64.7%)33759232/45929032 bytes (73.5%)37879808/45929032 bytes (82.5%)41926656/45929032 bytes (91.3%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root/.

Using TensorFlow backend.


In [ ]:
#Mounting the first drive:
from google.colab import drive
drive.mount('/content/drive1')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive1


In [ ]:
c_to_n={"Anomaly":0,"Normal":1}

In [ ]:
# Get model with pretrained weights.
base_model = InceptionV3(
weights='imagenet',
include_top=True)
    
    
# We'll extract features at the final pool layer.
model = Model(
    inputs=base_model.input, 
    outputs=base_model.get_layer('avg_pool').output)

96116736/96112376 [==============================] - 4s 0us/step


In [ ]:
def video_frame(path,label):
  
  features=[]
  images = []
  labels_t=[]
  cap = cv2.VideoCapture(path)
  count=0;
  framerate = int(cap.get(cv2.CAP_PROP_FPS))
  i=0;
  count_frame=0
  c=0
  while(True):
      # Capture frame-by-frame
      ret, frame = cap.read()

      if not ret: 
        break
      
#      if frame is None:
#       break;

      # Saves image of the current frame in jpg file
      
      elif count%(1*framerate) == 0 :
        frame = preprocess_input(cv2.resize(frame, (299,299)))
        images.append(frame)
        count_frame+=1;
        # cv2_imshow(frame)
      if(count_frame==8):
        f = model.predict(np.array(images))
        features.append(f)
        labels_t.append(c_to_n[label])
        images = []
        count_frame=0
      
      count+=1;
      
  # When everything done, release the capture
  cap.release()
  cv2.destroyAllWindows()
  
#   f = model.predict(np.array(images))
  
  return features, labels_t

In [ ]:
path=['/content/drive1/My Drive/Training_Set','/content/drive1/My Drive/Validation_Set','/content/drive1/My Drive/Testing_Set']

In [ ]:
labels=['Abuse','Arson','Assault','Burglary','Explosion','Fighting','RoadAccidents','Robbery','Shooting','Shoplifting','Stealing','Vandalism','Normal']
ignore=['.ipynb_checkpoints','Normal_Videos_251_x264.mp4']

In [ ]:
labels_train=[]
transfer_values_train=[]

p=path[0]
dir_list=os.listdir(p)
for dl in dir_list:
    print(str(dl))
    for lab in labels:
      if re.search(lab,str(dl))!=None:
        if lab=='Normal':
          label=lab
          break
        else:
          label='Anomaly'
          break
    features, l=video_frame(p+'/'+str(dl),label)
    labels_train.extend(l)
    print(np.array(features).shape)
    transfer_values_train.extend(np.array(features))
    print(np.array(transfer_values_train).shape)

    #flip the video
    if str(dl) not in ignore:
      clip = VideoFileClip(p+'/'+str(dl))
      reversed_clip = clip.fx(vfx.mirror_x)
      reversed_clip.write_videofile('new_video.mp4')
      features, l=video_frame('new_video.mp4',label)
      transfer_values_train.extend(np.array(features))
      labels_train.extend(l)
      print(np.array(transfer_values_train).shape)

Abuse005.mp4
(1, 8, 2048)
(1, 8, 2048)
[MoviePy] >>>> Building video new_video.mp4
[MoviePy] Writing audio in new_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 162/162 [00:00<00:00, 744.38it/s]

[MoviePy] Done.
[MoviePy] Writing video new_video.mp4



100%|██████████| 221/221 [00:00<00:00, 234.68it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: new_video.mp4 



...

[MoviePy] Done.
[MoviePy] >>>> Video ready: new_video.mp4 

(9502, 8, 2048)
Copy of Normal_Videos102_x264_Trim.mp4
(3, 8, 2048)
(9505, 8, 2048)
[MoviePy] >>>> Building video new_video.mp4
[MoviePy] Writing audio in new_videoTEMP_MPY_wvf_snd.mp3


100%|██████████| 684/684 [00:00<00:00, 980.12it/s]


[MoviePy] Done.
[MoviePy] Writing video new_video.mp4


100%|██████████| 930/930 [00:04<00:00, 223.64it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: new_video.mp4 

(9508, 8, 2048)


In [ ]:
labels_encoded_train = np_utils.to_categorical(labels_train)

In [ ]:
import pickle

pickle_out=open('/content/drive1/My Drive/Training_Set_values1.pickle',"wb")
pickle.dump(transfer_values_train, pickle_out)
pickle_out.close()

pickle_out=open('/content/drive1/My Drive/Training_Set_labels1.pickle',"wb")
pickle.dump(labels_encoded_train, pickle_out)
pickle_out.close()